In [ ]:

pip install gym-super-mario-bros
pip install tensordict
pip install torchrl

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import torch
from torch import nn
from torchvision import transforms as T
from PIL import Image
import numpy as np
from pathlib import Path
from collections import deque
import random, datetime, os

import gym
from gym.spaces import Box
from gym.wrappers import FrameStack

from nes_py.wrappers import JoypadSpace

import gym_super_mario_bros

from tensordict import TensorDict
from torchrl.data import TensorDictReplayBuffer, LazyMemmapStorage
import numpy as np
import time,datetime
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path


In [ ]:

if gym.__version__<'0.26':
    env=gym_super_mario_bros.make("SuperMarioBros-1-1-v0",new_step_api=True)
else:
    env=gym_super_mario_bros.make("SuperMarioBros-1-1-v0", render_mode='rgb', apply_api_compatibility=True)

env=JoypadSpace(env, [["right"], ["right", "A"]])

env.reset()
next_state, reward, done, trunc, info=env.step(action=0)
print(f"{next_state.shape},\n {reward},\n {done},\n {info}")

In [ ]:
class SkipFrame(gym.Wrapper):
    def __init__(self,env,skip):
        """Return only every `skip`-th frame"""
        super().__init__(env)
        self._skip=skip

    def step(self, action):
   
        total_reward=0.0
        for i in range(self._skip):
            # Accumulate reward and repeat the same action
            obs, reward, done, trunk, info = self.env.step(action)
            total_reward+=reward
            if done:
                break
        return obs, total_reward, done, trunk, info

In [ ]:
class GrayScaleObservation(gym.ObservationWrapper):
    def __init__(self,env):
        super().__init__(env)
        obs_shape=self.observation_space.shape[:2]
        self.observation_space=Box(low=0,high=255,shape=obs_shape,dtype=np.uint8)

    def permute_orientation(self, observation):
        observation=np.transpose(observation, (2, 0, 1))
        observation=torch.tensor(observation.copy(), dtype=torch.float)
        return observation

    def observation(self, observation):
        observation=self.permute_orientation(observation)
        transform=T.Grayscale()
        observation=transform(observation)
        return observation

In [ ]:
class ResizeObservation(gym.ObservationWrapper):
    def __init__(self, env, shape):
        super().__init__(env)
        if isinstance(shape, int):
            self.shape=(shape, shape)
        else:
            self.shape = tuple(shape)

        obs_shape=self.shape+self.observation_space.shape[2:]
        self.observation_space=Box(low=0, high=255, shape=obs_shape, dtype=np.uint8)

    def observation(self, observation):
        transforms=T.Compose(
            [T.Resize(self.shape, antialias=True), T.Normalize(0,255)]
        )
        observation = transforms(observation).squeeze(0)
        return observation

In [ ]:
class CustomReward(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)
        self.max_x = 0
        self.stuck_steps = 0

    def reset(self, **kwargs):
        self.max_x = 0
        self.stuck_steps = 0
        return self.env.reset(**kwargs)

    def step(self, action):
        state, reward, done, trunc, info = self.env.step(action)
        
        
        x_pos = info.get("x_pos", 0)

        
        if x_pos > self.max_x:
            self.max_x = x_pos
            self.stuck_steps = 0  
        else:
            self.stuck_steps += 1 

        if self.stuck_steps > 50:
            reward -= 1.0
            
       
        if done and not info.get("flag_get", False):
            reward -= 50
            
        reward -= 0.1 

        return state, reward, done, trunc, info

In [ ]:
env=SkipFrame(env, skip=4)
env=GrayScaleObservation(env)
env = CustomReward(env)
env=ResizeObservation(env, shape=84)
if gym.__version__ <'0.26':
    env=FrameStack(env, num_stack=4, new_step_api=True)
else:
    env=FrameStack(env, num_stack=4)

In [ ]:
class Mario:
    def __init__():
        pass

    def act(self,state):
        pass

    def cache(self,experience):
        pass

    def recall(self):
        pass

    def learn(self):
        pass
              

In [ ]:
class Mario:
    def __init__(self,state_dim,action_dim,save_dir):
        self.state_dim=state_dim
        self.action_dim=action_dim
        self.save_dir=save_dir

        self.device = "cuda" if torch.cuda.is_available() else "cpu"

        self.net=MarioNet(self.state_dim, self.action_dim).float()
        self.net=self.net.to(device=self.device)

        self.exploration_rate=1
        self.exploration_rate_decay=0.9999995 
        self.exploration_rate_min=0.1
        self.curr_step=0

        self.save_every=5e5 

    def act(self, state):
        """
    Given a state, choose an epsilon-greedy action and update value of step.

    Inputs:
    state(``LazyFrame``): A single observation of the current state, dimension is (state_dim)
    Outputs:
    ``action_idx`` (``int``): An integer representing which action Mario will perform
    """

        if np.random.rand()<self.exploration_rate:
            action_idx=np.random.randint(self.action_dim)

       
        else:
            state=state[0].__array__() if isinstance(state, tuple) else state.__array__()
            state=torch.tensor(state, device=self.device).unsqueeze(0)
            action_values=self.net(state, model="online")
            action_idx=torch.argmax(action_values, axis=1).item()

       
        self.exploration_rate *=self.exploration_rate_decay
        self.exploration_rate=max(self.exploration_rate_min, self.exploration_rate)

       
        self.curr_step +=1
        return action_idx

In [ ]:
class Mario(Mario):  
    def __init__(self, state_dim, action_dim, save_dir):
        super().__init__(state_dim, action_dim, save_dir)
        self.memory=TensorDictReplayBuffer(storage=LazyMemmapStorage(60000, device=torch.device("cpu")))
        self.batch_size=32

    def cache(self, state, next_state, action, reward, done):
        """
        Store the experience to self.memory (replay buffer)

        Inputs:
        state (``LazyFrame``),
        next_state (``LazyFrame``),
        action (``int``),
        reward (``float``),
        done(``bool``))
        """
        def first_if_tuple(x):
            return x[0] if isinstance(x, tuple) else x
        state=first_if_tuple(state).__array__()
        next_state=first_if_tuple(next_state).__array__()

        state=torch.tensor(state)
        next_state=torch.tensor(next_state)
        action=torch.tensor([action])
        reward=torch.tensor([reward])
        done=torch.tensor([done])

       
        self.memory.add(TensorDict({"state": state, "next_state": next_state, "action": action, "reward": reward, "done": done}, batch_size=[]))

    def recall(self):
        """
        Retrieve a batch of experiences from memory
        """
        batch=self.memory.sample(self.batch_size).to(self.device)
        state, next_state, action, reward, done = (batch.get(key) for key in ("state", "next_state", "action", "reward", "done"))
        return state, next_state, action.squeeze(), reward.squeeze(), done.squeeze()

In [ ]:
class MarioNet(nn.Module):
    """mini CNN structure
  input -> (conv2d + relu) x 3 -> flatten -> (dense + relu) x 2 -> output
  """

    def __init__(self, input_dim, output_dim):
        super().__init__()
        c, h, w=input_dim

        if h!=84:
            raise ValueError(f"Expecting input height: 84, got: {h}")
        if w!=84:
            raise ValueError(f"Expecting input width: 84, got: {w}")

        self.online=self.__build_cnn(c, output_dim)

        self.target=self.__build_cnn(c, output_dim)
        self.target.load_state_dict(self.online.state_dict())

        
        for p in self.target.parameters():
            p.requires_grad=False

    def forward(self, input, model):
        if model=="online":
            return self.online(input)
        elif model=="target":
            return self.target(input)

    def __build_cnn(self, c, output_dim):
        return nn.Sequential(
            nn.Conv2d(in_channels=c, out_channels=32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(3136, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim),
        )

In [ ]:
class Mario(Mario):
    def __init__(self, state_dim, action_dim, save_dir):
        super().__init__(state_dim, action_dim, save_dir)
        self.gamma=0.9

    def td_estimate(self, state, action):
        current_Q=self.net(state, model="online")[
            np.arange(0, self.batch_size), action
        ] 
        return current_Q

    @torch.no_grad()
    def td_target(self, reward, next_state, done):
        next_state_Q=self.net(next_state, model="online")
        best_action=torch.argmax(next_state_Q, axis=1)
        next_Q=self.net(next_state, model="target")[
            np.arange(0, self.batch_size), best_action
        ]
        return (reward +(1 - done.float()) * self.gamma * next_Q).float()

In [ ]:
class Mario(Mario):
    def __init__(self, state_dim, action_dim, save_dir):
        super().__init__(state_dim, action_dim, save_dir)
        self.optimizer=torch.optim.Adam(self.net.parameters(), lr=0.00025)
        self.loss_fn=torch.nn.SmoothL1Loss()

    def update_Q_online(self, td_estimate, td_target):
        loss=self.loss_fn(td_estimate, td_target)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss.item()

    def sync_Q_target(self):
        self.net.target.load_state_dict(self.net.online.state_dict())

In [ ]:
class Mario(Mario):
    def save(self):
        save_path=(self.save_dir / f"mario_net_{int(self.curr_step // self.save_every)}.chkpt")
        torch.save(
            dict(model=self.net.state_dict(), exploration_rate=self.exploration_rate),
            save_path,
        )
        print(f"MarioNet saved to {save_path} at step {self.curr_step}")

In [ ]:
class Mario(Mario):
    def __init__(self, state_dim, action_dim, save_dir):
        super().__init__(state_dim, action_dim, save_dir)
        self.burnin=1e4  
        self.learn_every=3 
        self.sync_every=1e4 

    def learn(self):
        if self.curr_step % self.sync_every==0:
            self.sync_Q_target()

        if self.curr_step % self.save_every==0:
            self.save()

        if self.curr_step < self.burnin:
            return None, None

        if self.curr_step % self.learn_every!=0:
            return None, None

       
        state, next_state, action, reward, done=self.recall()
        
        td_est=self.td_estimate(state, action)

        td_tgt=self.td_target(reward, next_state, done)

        loss=self.update_Q_online(td_est, td_tgt)

        return (td_est.mean().item(), loss)

In [ ]:


class MetricLogger:
    def __init__(self, save_dir: Path):
        save_dir.mkdir(parents=True, exist_ok=True)

        self.save_log = save_dir / "log"
        
        if not self.save_log.exists():
            with open(self.save_log, "w") as f:
                f.write(
                    f"{'Episode':>8}{'Step':>8}{'Epsilon':>10}{'MeanReward':>15}"
                    f"{'MeanLength':>15}{'MeanLoss':>15}{'MeanQValue':>15}"
                    f"{'TimeDelta':>15}{'Time':>20}\n"
                )
        
     
        self.ep_rewards = []
        self.ep_lengths = []
        self.ep_avg_losses = []
        self.ep_avg_qs = []

        
        self.moving_avg_ep_rewards = []
        self.moving_avg_ep_lengths = []
        self.moving_avg_ep_avg_losses = []
        self.moving_avg_ep_avg_qs = []

        self.init_episode()
        self.record_time = time.time()

    def init_episode(self):
        """Reset current episode stats."""
        self.curr_ep_reward = 0.0
        self.curr_ep_length = 0
        self.curr_ep_loss = 0.0
        self.curr_ep_q = 0.0
        self.curr_ep_loss_length = 0

    def log_step(self, reward, loss, q):
        """
        Call this at every environment step.
        - reward: scalar reward from env
        - loss: training loss (or None if no update done on this step)
        - q: mean Q-value for action (or None if not computed)
        """
        self.curr_ep_reward += reward
        self.curr_ep_length += 1

        # Use "is not None" so we don't skip loss = 0.0
        if loss is not None:
            self.curr_ep_loss += float(loss)
            if q is not None:
                self.curr_ep_q += float(q)
            self.curr_ep_loss_length += 1

    def log_episode(self):
        """Call once at the end of an episode."""
        self.ep_rewards.append(self.curr_ep_reward)
        self.ep_lengths.append(self.curr_ep_length)

        if self.curr_ep_loss_length == 0:
            ep_avg_loss = 0.0
            ep_avg_q = 0.0
        else:
            ep_avg_loss = np.round(self.curr_ep_loss / self.curr_ep_loss_length, 5)
            ep_avg_q = np.round(self.curr_ep_q / self.curr_ep_loss_length, 5)

        self.ep_avg_losses.append(ep_avg_loss)
        self.ep_avg_qs.append(ep_avg_q)

        self.init_episode()

    def record(self, episode, epsilon, step, log_to_console=False):
        """
        Call once per episode AFTER log_episode().
        - If log_to_console is True, prints summary.
        - Always writes to file.
        """
        mean_ep_reward = np.round(np.mean(self.ep_rewards[-100:]), 3)
        mean_ep_length = np.round(np.mean(self.ep_lengths[-100:]), 3)
        mean_ep_loss = np.round(np.mean(self.ep_avg_losses[-100:]), 3)
        mean_ep_q = np.round(np.mean(self.ep_avg_qs[-100:]), 3)
        
        self.moving_avg_ep_rewards.append(mean_ep_reward)
        self.moving_avg_ep_lengths.append(mean_ep_length)
        self.moving_avg_ep_avg_losses.append(mean_ep_loss)
        self.moving_avg_ep_avg_qs.append(mean_ep_q)

        last_record_time = self.record_time
        self.record_time = time.time()
        time_since_last_record = np.round(self.record_time - last_record_time, 3)

        if log_to_console:
            print(
                f"Episode {episode} - "
                f"Step {step} - "
                f"Epsilon {epsilon:.4f} - "
                f"Mean Reward {mean_ep_reward} - "
                f"Mean Length {mean_ep_length} - "
                f"Mean Loss {mean_ep_loss} - "
                f"Mean Q Value {mean_ep_q} - "
                f"Time Delta {time_since_last_record} - "
                f"Time {datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')}"
            )

        with open(self.save_log, "a") as f:
            f.write(
                f"{episode:8d}{step:8d}{epsilon:10.3f}"
                f"{mean_ep_reward:15.3f}{mean_ep_length:15.3f}{mean_ep_loss:15.3f}{mean_ep_q:15.3f}"
                f"{time_since_last_record:15.3f}"
                f"{datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S'):>20}\n"
            )


In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print("Using device:", device)

# TRAINING

In [ ]:
import torch
from pathlib import Path
import gc


use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print("Using device:", device)

save_dir = Path("/kaggle/working")

metric_log_dir = save_dir / "log"
metric_logger = MetricLogger(metric_log_dir)


mario = Mario(
    state_dim=(4, 84, 84),
    action_dim=env.action_space.n,
    save_dir=save_dir
)

episodes = 8000

log_file = save_dir / "training_log.txt"

print(f"Starting Training for {episodes} episodes...")

for e in range(episodes):
    state = env.reset()
    ep_reward = 0.0

    metric_logger.init_episode()

    done = False
    while not done:
        action = mario.act(state)

        step_result = env.step(action)
        if len(step_result) == 5:
            next_state, reward, terminated, truncated, info = step_result
            done = terminated or truncated
        else:
            next_state, reward, done, info = step_result

        mario.cache(state, next_state, action, reward, done)

        q, loss = mario.learn()

        metric_logger.log_step(reward, loss, q)

        ep_reward += reward
        state = next_state

        if info.get("flag_get", False):
            break

    metric_logger.log_episode()
    metric_logger.record(
        episode=e,
        epsilon=mario.exploration_rate,
        step=mario.curr_step,
        log_to_console=False     
    )

    if e % 20 == 0:
        print(f"Episode {e}/{episodes} | Step {mario.curr_step} | "
              f"Reward {ep_reward:.1f} | Epsilon {mario.exploration_rate:.4f}")
        
        with open(log_file, "a") as f:
            f.write(f"{e},{mario.curr_step},{ep_reward},{mario.exploration_rate}\n")

        if e > 0 and e % 50 == 0:
            save_path = save_dir / "mario_net.pth"
            torch.save(mario.net.state_dict(), save_path)

        if e > 0 and e % 500 == 0:
            torch.save(mario.net.state_dict(), save_dir / f"mario_ep_{e}.pth")

        if use_cuda:
            torch.cuda.empty_cache()
        gc.collect()


In [ ]:
import torch
torch.save(mario.net.state_dict(), "mario_net_checkpoint.pth")
print("Saved mario_net_checkpoint.pth successfully!")

## TESTING

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import imageio
import warnings
from torchvision import transforms as T
from gym.spaces import Box
from gym.wrappers import FrameStack
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from nes_py.wrappers import JoypadSpace
import gym

warnings.filterwarnings('ignore')


class SkipFrame(gym.Wrapper):
    def __init__(self, env, skip):
        super().__init__(env)
        self._skip = skip
    def step(self, action):
        total_reward = 0.0
        for i in range(self._skip):
            obs, reward, done, trunc, info = self.env.step(action)
            total_reward += reward
            if done: break
        return obs, total_reward, done, trunc, info

class GrayScaleObservation(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        obs_shape = self.observation_space.shape[:2]
        self.observation_space = Box(low=0, high=255, shape=obs_shape, dtype=np.uint8)
    def permute_orientation(self, observation):
        observation = np.transpose(observation, (2, 0, 1))
        observation = torch.tensor(observation.copy(), dtype=torch.float)
        return observation
    def observation(self, observation):
        observation = self.permute_orientation(observation)
        transform = T.Grayscale()
        observation = transform(observation)
        return observation

class ResizeObservation(gym.ObservationWrapper):
    def __init__(self, env, shape):
        super().__init__(env)
        self.shape = (shape, shape) if isinstance(shape, int) else tuple(shape)
        obs_shape = self.shape + self.observation_space.shape[2:]
        self.observation_space = Box(low=0, high=255, shape=obs_shape, dtype=np.uint8)
    def observation(self, observation):
        transforms = T.Compose([T.Resize(self.shape, antialias=True), T.Normalize(0, 255)])
        observation = transforms(observation).squeeze(0)
        return observation

# 2. Model
class MarioNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        c, h, w = input_dim
        self.online = nn.Sequential(
            nn.Conv2d(in_channels=c, out_channels=32, kernel_size=8, stride=4), nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2), nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1), nn.ReLU(),
            nn.Flatten(),
            nn.Linear(3136, 512), nn.ReLU(),
            nn.Linear(512, output_dim),
        )
    def forward(self, input, model):
        return self.online(input)

class Mario:
    def __init__(self, state_dim, action_dim):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.net = MarioNet(self.state_dim, self.action_dim).float().to(self.device)
        self.exploration_rate = 0.0

    def act(self, state):
        state = state[0].__array__() if isinstance(state, tuple) else state.__array__()
        state = torch.tensor(state, device=self.device).unsqueeze(0)
        action_values = self.net(state, model="online")
        return torch.argmax(action_values, axis=1).item()

# 3. Setup
if gym.__version__ < '0.26':
    env = gym_super_mario_bros.make("SuperMarioBros-1-1-v0", new_step_api=True)
else:
    env = gym_super_mario_bros.make("SuperMarioBros-1-1-v0", render_mode='rgb', apply_api_compatibility=True)
env = JoypadSpace(env, [["right"], ["right", "A"]])
env = SkipFrame(env, skip=4)
env = GrayScaleObservation(env)
env = ResizeObservation(env, shape=84)
if gym.__version__ < '0.26':
    env = FrameStack(env, num_stack=4, new_step_api=True)
else:
    env = FrameStack(env, num_stack=4)

mario = Mario(state_dim=(4, 84, 84), action_dim=env.action_space.n)


print("Loading weights...")
try:
    checkpoint = torch.load("/kaggle/working/mario_net_checkpoint.pth", map_location="cpu")
    
    mario.net.load_state_dict(checkpoint, strict=False)
    
    mario.net.eval()
    print(" Weights loaded ")
except FileNotFoundError:
    print("Checkpoint not found. Running with random weights.")
except Exception as e:
    print(f" An unexpected error occurred: {e}")

# 5. Record Long Video
print(" Recording 1 minute of gameplay ")
frames = []
state = env.reset()
if isinstance(state, tuple): state = state[0]

for i in range(4000): 
    try:
        frame = env.unwrapped.render(mode='rgb_array')
    except:
        frame = env.render()
        
    if frame is not None:
        frames.append(np.copy(frame))
    
    action = mario.act(state)
    step_result = env.step(action)
    
    if len(step_result) == 5:
        next_state, reward, done, trunc, info = step_result
    else:
        next_state, reward, done, info = step_result
        
    state = next_state
    
    if i % 200 == 0:
        print(f"Captured {i} frames...")

    if done: 
        print(f"Mario died/finished at frame {i}")
        break

env.close()

if len(frames) > 0:
    print(f"Saving {len(frames)} frames to 'mario_long.gif'...")
    imageio.mimsave('mario_long.gif', frames, fps=25)
    print(" Run the display cell below to view.")
else:
    print(" Error: No frames were captured.")

In [ ]:
from IPython.display import Image
Image(filename='mario_long.gif')

#  **GRAPH**

In [ ]:
log_path = "/kaggle/working/training_log.txt"

df = pd.read_csv(
    log_path,
    names=["episode", "step", "reward", "epsilon"])

plt.figure(figsize=(10, 5))
plt.plot(df["episode"], df["reward"])
plt.xlabel("Episode")
plt.ylabel("Episode Reward")
plt.title("Mario Training: Episode vs Reward")
plt.grid(True)
plt.show()


In [ ]:
window = 50 
df["reward_ma"] = df["reward"].rolling(window=window).mean()

plt.figure(figsize=(10, 5))
plt.plot(df["episode"], df["reward"], alpha=0.3, label="Reward (per episode)")
plt.plot(df["episode"], df["reward_ma"], label=f"Reward {window}-ep Moving Avg")
plt.xlabel("Episode")
plt.ylabel("Reward")
plt.title("Mario Training: Episode Reward and Moving Average")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
df["steps_per_episode"] = df["step"].diff().fillna(df["step"])
plt.figure(figsize=(10, 5))
plt.plot(df["episode"], df["steps_per_episode"])
plt.xlabel("Episode")
plt.ylabel("Steps in Episode (approx)")
plt.title("Approx. Steps per Episode")
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(df["epsilon"], df["reward"], alpha=0.4)
plt.xlabel("Epsilon")
plt.ylabel("Reward")
plt.title("Reward vs Epsilon")
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(8, 6))
plt.hist(df["reward"], bins=30)
plt.xlabel("Reward")
plt.ylabel("Count")
plt.title("Distribution of Episode Rewards")
plt.grid(True)
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

metric_log_path = "/kaggle/working/log/log"

dfm = pd.read_csv(
    metric_log_path,
    delim_whitespace=True
)

print(dfm.head()) 

plt.figure(figsize=(10, 5))
plt.plot(dfm["Episode"], dfm["MeanReward"])
plt.xlabel("Episode")
plt.ylabel("Mean Reward (last 100)")
plt.title("Moving Average Reward (MetricLogger)")
plt.grid(True)
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(dfm["Episode"], dfm["MeanLoss"])
plt.xlabel("Episode")
plt.ylabel("Mean Loss")
plt.title("Mean Loss per Episode")
plt.grid(True)
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(dfm["Episode"], dfm["MeanQValue"])
plt.xlabel("Episode")
plt.ylabel("Mean Q-Value")
plt.title("Mean Q-Value per Episode")
plt.grid(True)
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(dfm["Episode"], dfm["MeanLength"])
plt.xlabel("Episode")
plt.ylabel("Mean Episode Length")
plt.title("Mean Episode Length per Episode")
plt.grid(True)
plt.show()


# **CONTINUATION TRAINING**

In [ ]:
checkpoint_path="/kaggle/working/mario_net_checkpoint.pth"   

print(" Loading previous checkpoint...")

try:
    mario.net.load_state_dict(torch.load(checkpoint_path, map_location=device), strict=False)
    print(" Checkpoint loaded. Continuing training...")
except Exception as e:
    print(" Could not load checkpoint:", e)


In [ ]:
save_dir = Path("/kaggle/working")
log_file = save_dir / "training_log.txt"
mario = Mario(state_dim=(4, 84, 84), action_dim=env.action_space.n, save_dir=save_dir)

In [ ]:
mario.exploration_rate=0.4773

In [ ]:
episodes = 8000 
more_episodes = 6000 
print(f"🚀 Continuing Training for {more_episodes} additional episodes...")

for e in range(episodes, episodes + more_episodes):  
    state = env.reset()
    ep_reward = 0

    while True:
        action = mario.act(state)
        step_result = env.step(action)

        if len(step_result) == 5:
            next_state, reward, done, trunc, info = step_result
        else:
            next_state, reward, done, info = step_result

        mario.cache(state, next_state, action, reward, done)
        q, loss = mario.learn()

        ep_reward += reward
        state = next_state

        if done or info.get("flag_get", False):
            break

    if e % 20 == 0:
        print(f"Episode {e} | Step {mario.curr_step} | Reward {ep_reward:.1f} | Epsilon {mario.exploration_rate:.4f}")
        with open(log_file, "a") as f:
            f.write(f"{e},{mario.curr_step},{ep_reward},{mario.exploration_rate}\n")

    if e % 50 == 0:
        torch.save(mario.net.state_dict(), save_dir / "mario_net.pth")

    if e % 500 == 0:
        torch.save(mario.net.state_dict(), save_dir / f"mario_ep_{e}.pth")


In [ ]:

torch.save(mario.net.state_dict(), "mario_net_checkpoint.pth")
print("Saved mario_net_checkpoint.pth successfully!")

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import imageio
import warnings
from torchvision import transforms as T
from gym.spaces import Box
from gym.wrappers import FrameStack
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from nes_py.wrappers import JoypadSpace
import gym

warnings.filterwarnings('ignore')

# 1. Wrappers
class SkipFrame(gym.Wrapper):
    def __init__(self, env, skip):
        super().__init__(env)
        self._skip = skip
    def step(self, action):
        total_reward = 0.0
        for i in range(self._skip):
            obs, reward, done, trunc, info = self.env.step(action)
            total_reward += reward
            if done: break
        return obs, total_reward, done, trunc, info

class GrayScaleObservation(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        obs_shape = self.observation_space.shape[:2]
        self.observation_space = Box(low=0, high=255, shape=obs_shape, dtype=np.uint8)
    def permute_orientation(self, observation):
        observation = np.transpose(observation, (2, 0, 1))
        observation = torch.tensor(observation.copy(), dtype=torch.float)
        return observation
    def observation(self, observation):
        observation = self.permute_orientation(observation)
        transform = T.Grayscale()
        observation = transform(observation)
        return observation

class ResizeObservation(gym.ObservationWrapper):
    def __init__(self, env, shape):
        super().__init__(env)
        self.shape = (shape, shape) if isinstance(shape, int) else tuple(shape)
        obs_shape = self.shape + self.observation_space.shape[2:]
        self.observation_space = Box(low=0, high=255, shape=obs_shape, dtype=np.uint8)
    def observation(self, observation):
        transforms = T.Compose([T.Resize(self.shape, antialias=True), T.Normalize(0, 255)])
        observation = transforms(observation).squeeze(0)
        return observation

# 2. Model
class MarioNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        c, h, w = input_dim
        self.online = nn.Sequential(
            nn.Conv2d(in_channels=c, out_channels=32, kernel_size=8, stride=4), nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2), nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1), nn.ReLU(),
            nn.Flatten(),
            nn.Linear(3136, 512), nn.ReLU(),
            nn.Linear(512, output_dim),
        )
    def forward(self, input, model):
        return self.online(input)

class Mario:
    def __init__(self, state_dim, action_dim):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.net = MarioNet(self.state_dim, self.action_dim).float().to(self.device)
        self.exploration_rate = 0.0

    def act(self, state):
        state = state[0].__array__() if isinstance(state, tuple) else state.__array__()
        state = torch.tensor(state, device=self.device).unsqueeze(0)
        action_values = self.net(state, model="online")
        return torch.argmax(action_values, axis=1).item()


if gym.__version__ < '0.26':
    env = gym_super_mario_bros.make("SuperMarioBros-1-1-v0", new_step_api=True)
else:
    env = gym_super_mario_bros.make("SuperMarioBros-1-1-v0", render_mode='rgb', apply_api_compatibility=True)
env = JoypadSpace(env, [["right"], ["right", "A"]])
env = SkipFrame(env, skip=4)
env = GrayScaleObservation(env)
env = ResizeObservation(env, shape=84)
if gym.__version__ < '0.26':
    env = FrameStack(env, num_stack=4, new_step_api=True)
else:
    env = FrameStack(env, num_stack=4)

mario = Mario(state_dim=(4, 84, 84), action_dim=env.action_space.n)


print("Loading weights...")
try:
    checkpoint = torch.load("/kaggle/working/mario_net_checkpoint.pth", map_location="cpu")
    
    
    mario.net.load_state_dict(checkpoint, strict=False)
    
    mario.net.eval()
    print(" Weights loaded! (Ignored unnecessary 'target' layers)")
except FileNotFoundError:
    print("⚠️ Checkpoint not found. Running with random weights.")
except Exception as e:
    print(f" An unexpected error occurred: {e}")

# 5. Record Long Video
print("🎥 Recording 1 minute of gameplay (approx 2000 frames)...")
frames = []
state = env.reset()
if isinstance(state, tuple): state = state[0]

for i in range(4000): 
    try:
        frame = env.unwrapped.render(mode='rgb_array')
    except:
        frame = env.render()
        
    if frame is not None:
        frames.append(np.copy(frame))
    
    action = mario.act(state)
    step_result = env.step(action)
    
    if len(step_result) == 5:
        next_state, reward, done, trunc, info = step_result
    else:
        next_state, reward, done, info = step_result
        
    state = next_state
    
    if i % 200 == 0:
        print(f"Captured {i} frames...")

    if done: 
        print(f"Mario died/finished at frame {i}")
        break

env.close()

if len(frames) > 0:
    print(f" Saving {len(frames)} frames to 'mario_long.gif'...")
    imageio.mimsave('mario_long.gif', frames, fps=25)
    print(" Done! Run the display cell below to view.")
else:
    print(" Error: No frames were captured.")

In [ ]:
from IPython.display import Image
Image(filename='/kaggle/working/mario_long.gif')